In [1]:
import numpy as np

import matplotlib.pyplot as plt
from scipy import integrate
from scipy import linalg


N_sp = 50    ### Number of species

D = 100000   ### Diffusion coefficient

n_sp_full = []

habitats = np.array([(i+1)**2 for i in range(5)])   ## Number of patches

spatial_corr = 0           ## Spatial correlation between patches



## Function to get a discrete Laplacian for a matrix of species densities over all patches:

In [2]:
def discrete_laplacian(M):
    L = -4*M
    L += np.roll(M, (0,-1), (0,1)) # right neighbor
    L += np.roll(M, (0,+1), (0,1)) # left neighbor
    L += np.roll(M, (-1,0), (0,1)) # top neighbor
    L += np.roll(M, (+1,0), (0,1)) # bottom neighbor
    
    return L


## The function f(t,y) feeds into the ODE solver using a discretization of the Laplacian.
Note that low densities ($< 10^{-5}$) must be set to 0 before every step. This prevents exponential growth from resurrecting extinct species at a given patch. The only channel of rescue for that species comes from dispersal from other patches.

In [3]:
def f(t,y):
    
    y = [y[i] if y[i] > 10**-5 else 0 for i in range(len(y))]  ### Low densities must be set to 0 before every step. 
    
    y_diff = np.copy(y)
    
    for i in range(N_sp):
        sp_mat_temp = np.array( [ [ y[(j+k*int(np.sqrt(ii)))*N_sp + i] for j in range(int(np.sqrt(ii)))] for k in range(int(np.sqrt(ii))) ] )
        lap = discrete_laplacian(sp_mat_temp)
        for k in range(int(np.sqrt(ii))):
            for j in range(int(np.sqrt(ii))):
                y_diff[(j+k*int(np.sqrt(ii)))*N_sp + i] = lap[k,j]
                
        

    return np.multiply(y,np.tile(r,ii)  + np.dot( B ,y ) - np.dot( np.tile( r/K, ii )*np.eye(N_sp*ii), y ) ) + D*y_diff



## The routine below can be tweaked to incorporate different simulation settings.
Note: The standard deviation used below is above May's limit as in Fig. 2. Hence, the code takes longer to run than usual. Also, the spatial autocorrelation is 0. This can be easily changed through the spatial_corr variable in the first code cell. The a_rand variable below easily generates interaction matrices for different patches based on the covariance matrix  cov_mat.

In [ ]:
for ii in habitats:

    K = np.random.normal(1, 0.0, N_sp) ### Carrying capacities

    r = np.random.normal(1, 0.0, N_sp) ### Growth rates


        ####### Multiple interaction matrices for different patches ########
    
    patch_means = np.array([0 for i in range(ii)])  

    patch_std = (5/4)/np.sqrt(N_sp)  

    
    
    cov_mat = np.array([ [patch_std**2 if i==j else (patch_std**2)*spatial_corr for j in range(ii)] for i in range(ii) ])

    rng = np.random.default_rng()

    a_rand = rng.multivariate_normal(patch_means, cov_mat, (N_sp, N_sp)).T


    B = np.array([ [ 0.0 if j==k or (k//N_sp != j//N_sp) else a_rand[j//(N_sp), j%N_sp, k%N_sp] for k in range(N_sp*ii) ] for j in range(N_sp*ii) ])

    
    sol = integrate.solve_ivp(f, [0,100], [1.1 for j in range(N_sp*ii)])
    
    n_sp_full.append( len( [sol.y[j, len(sol.t) - 1] for j in range(N_sp) if sol.y[j, len(sol.t) - 1] > 10**-5] ) )  ### All patches have the same number of species in spatially coherent limit
    

plt.scatter(habitats, n_sp_full/N_sp)

plt.xlabel('Number of habitats', fontsize=20)
plt.ylabel('Relative species richness', fontsize=20)
plt.show()